In [10]:
%pip install -U langchain_openai


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: C:\Users\nbogd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/49.6 kB ? eta -:--:--
   ---------------------------------------- 49.6/49.6 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain_openai
    Found existing installation: langchain-openai 0.1.23
    Uninstalling langchain-openai-0.1.23:
      Successfully uninstalled langchain-openai-0.1.23


In [1]:
import os
import getpass
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from fpdf import FPDF
import datetime
os.environ["OPENAI_API_KEY"] = getpass.getpass()

C:\Users\nbogd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
file_path = "./news.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

print(f"Loaded {len(pages)} pages from the PDF file")

# Keywords to search for in the PDF
keywords = ["RSG", "CMM.AU", "EMR.AU", "GMD.AU", "EVN.AU", "GOR.AU", "NEM.AU", "NST.AU", "PRU.AU", "RMS.AU", "RED.AU", "RRL.AU", "WAF.AU", "WGX.AU", "BGL.AU", "gold", "Gold"]
print(f"Searching for the following keywords: {keywords}")

# Filter the pages based on the keywords
filtered_pages_ids = []
filtered_pages = []
for index in range(0, len(pages), 1):
    if any(keyword in pages[index].page_content for keyword in keywords):
        filtered_pages.append(pages[index])
        filtered_pages_ids.append(index)

print(f"Found {len(filtered_pages)} pages containing the keywords")

Loaded 252 pages from the PDF file
Searching for the following keywords: ['RSG', 'CMM.AU', 'EMR.AU', 'GMD.AU', 'EVN.AU', 'GOR.AU', 'NEM.AU', 'NST.AU', 'PRU.AU', 'RMS.AU', 'RED.AU', 'RRL.AU', 'WAF.AU', 'WGX.AU', 'BGL.AU', 'gold', 'Gold']
Found 104 pages containing the keywords


In [68]:
current_year = datetime.datetime.now().year

class DateExtractor(BaseModel):
    date: str = Field(description=f"Date in {current_year}/MM/DD format", example="2022/01/01")

model = ChatOpenAI(model="gpt-4o", temperature=0)
get_date_llm = model.with_structured_output(DateExtractor)

regex = "[A-Z][a-z]*, [A-Z][a-z]* [0-2]?[0-9]"

In [89]:
import re

dates = [None] * len(pages)

def get_date(index: int):
    if dates[index] != None:
        return dates[index]
    if index < 0:
        return None
    found = re.search(regex, pages[index].page_content[:150])
    if found is not None:
        date = found.group()
        date_string = get_date_llm.invoke(date).date
        datetimeObj = datetime.datetime.strptime(date_string, "%Y/%m/%d")
        dates[index] = datetimeObj
        return datetimeObj
    else:
        from_other = get_date(index-1)
        dates[index] = from_other
        return from_other
    
for index in filtered_pages_ids:
    get_date(index)


___________________________________
Friday, June 28
2024/06/28
2024-06-28 00:00:00
___________________________________
___________________________________
Friday, June 28
2024/06/28
2024-06-28 00:00:00
___________________________________


In [90]:
for date in dates:
    print(date)

None
2024-06-25 00:00:00
2024-07-22 00:00:00
2024-07-22 00:00:00
None
None
None
None
2024-07-21 00:00:00
None
None
None
2024-07-21 00:00:00
None
None
None
2024-07-19 00:00:00
2024-07-19 00:00:00
2024-07-19 00:00:00
None
None
None
None
2024-07-19 00:00:00
2024-07-19 00:00:00
None
None
None
None
None
2024-07-19 00:00:00
2024-07-18 00:00:00
2024-07-18 00:00:00
None
2024-07-18 00:00:00
2024-07-18 00:00:00
None
None
None
None
2024-07-18 00:00:00
2024-07-17 00:00:00
None
None
2024-07-17 00:00:00
None
2024-07-17 00:00:00
2024-07-17 00:00:00
2024-07-17 00:00:00
None
None
2024-07-17 00:00:00
None
None
2024-07-16 00:00:00
None
2024-07-16 00:00:00
2024-07-16 00:00:00
2024-07-16 00:00:00
None
None
2024-07-16 00:00:00
None
2024-07-15 00:00:00
2024-07-15 00:00:00
None
2024-07-15 00:00:00
2024-07-15 00:00:00
None
None
2024-07-15 00:00:00
2024-07-14 00:00:00
None
2024-07-14 00:00:00
None
None
None
2024-07-12 00:00:00
Friday, July 12
Friday, July 12
None
None
None
2024-07-12 00:00:00
2024-07-12 00:00:0

## For all

In [74]:
keywords_text = ", ".join(keywords)

# You can the prompt template if you feel something is off
# {} are placeholders for the variables that will be filled in

llm = ChatOpenAI(model="gpt-4o", temperature=0)
extractor_prompt = PromptTemplate.from_template("""You are a helpful assistant that extracts news about the stock market and gives gold-related information.
Include only the information connected to these keywords or some variation of them: {keywords} and information about funds and indexes. DO NOT analyze the information just list it as it is. DO NOT change or include new information. Use only the given sources. Use bullet points but include all information and extract information which pertains to the {keywords}. At the top state which day is the info from.
I want you to extract exhaustively every entry that has the {keywords}.
 

Capture All General News and Research House Recommendations connected with gold or the {keywords}.
Capture all news related to stock price movements (e.g., leadership transitions, mergers) as well as research house recommendations.
Ensure the extraction is not limited to research house recommendations.
Example: Include mentions like "RED.AU leadership transition after a merger with Silver Lake" alongside typical stock recommendations.

Link Price Movements to Gold Sector Trends and Broader Market Conditions
Ensure stock price movements are tied to gold sector trends or broader market conditions, such as demand for gold or inflation hedges.
Example: If a stock (e.g., RSG.AU) is mentioned alongside a gold price movement (e.g., "Gold lifted to $2,355"), link both the stock movement and the gold sector update.

Here is the information to analize:
{date}
{news}
""")
extractor = extractor_prompt | llm

# Extract the news from the filtered pages
news_per_page = []
for index in range(0, len(filtered_pages), 1):
    curr_date = dates[filtered_pages_ids[index]]
    if (type(curr_date) == type(datetime.date)):
        date_string = curr_date.strftime("%Y/%m/%d")
    else:
        date_string = curr_date
    page_analysis = extractor.invoke({
        "keywords": keywords_text,
        "date": date_string,
        "news": filtered_pages[index].page_content
    })
    news_per_page.append(page_analysis)

print(f"Extracted the news from the filtered pages")

Extracted the news from the filtered pages


## Combine dates and news

In [75]:
news_with_date = []
for index in range(0, len(filtered_pages), 1):
    news_with_date.append({
        "date": dates[filtered_pages_ids[index]],
        "content": news_per_page[index].content
    })

## Sort by date

In [ ]:
news_with_date.sort(key=lambda x: x["date"], reverse=True)

## Group by date

In [111]:
news_grouped_by_date = {}
for news_item in news_with_date:
    if news_item["date"] not in news_grouped_by_date:
        news_grouped_by_date[news_item["date"]] = []
    news_grouped_by_date[news_item["date"]].append(news_item["content"])

In [112]:
news_texts = []
for date, news in news_grouped_by_date.items():
    date_string = date.strftime("%B %d")
    news_texts.append(f"Date: {date_string}\n\n" + "\n\n".join(news))

In [84]:
summarizer_prompt = PromptTemplate.from_template("""You are a helpful assistant that gives reports about stock market news related to gold. 
By given list of news group them by date.
Use bullet points and include all information. Do not change or include new information.
{news}
""")
llm = ChatOpenAI(model="gpt-4o", temperature=0)
summarizer = summarizer_prompt | llm

# Call the summarizer

# final = summarizer.invoke({
#     "news": news_text
# })


In [114]:
finals = []
for news_text in news_texts:
    final = summarizer.invoke({
        "news": news_text
    })
    finals.append(final)

In [88]:
print(finals)

content="**Date: Monday, July 22, 2024**\n\n- Australian equities ended lower, with the ASX-200 falling (39.9) pts or (0.50%) to close at 7931.7.\n- The market saw broad-based selling, with 9 of 11 sectors ending in the red.\n- Energy was the weakest sector following a sharp fall in WTI Crude. Resource stocks were also soft, with lithium miners being the worst performers.\n- Gold remained steady, having retreated back into the prior three-month range.\n- Commodities: WTI crude recovered from one-month lows; copper fell to three-month lows due to concerns around demand in China; Singapore iron ore drifted lower for a third session amid news that China's Third Plenary policy meeting concluded without indications of short-term stimulus." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 224, 'total_tokens': 391, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name':

In [115]:
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Gold-Related Stock Market News', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(10)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

pdf = PDF()
for final in finals:
    pdf.add_page()
    pdf.chapter_body(final.content)

date = datetime.datetime.now().strftime("%Y-%m-%d")
output_path = f"PDFEXtract gold_news_summary_{date}.pdf"
pdf.output(output_path)

print(f"Saved the summarized news to {output_path}")

Saved the summarized news to PDFEXtract gold_news_summary_2024-10-02.pdf
